In [15]:
import openslide
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import ast
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

In [16]:
df = pd.read_csv('../input/mayo-clinic-strip-ai/train.csv')

In [17]:
df['label'] = df['label'].apply(lambda x: 1 if x == 'CE' else 0)

In [18]:
df = df.set_index('image_id')

In [19]:
samples = pd.read_csv('../input/mayo-clinic-strip-ai/test.csv')

In [20]:
final_data= pd.read_csv('../input/pathnavv2/navigation_v2.csv')
final_data = final_data.set_index('Unnamed: 0')
final_data = final_data.astype(object)

In [21]:
def get_patch(path, tl_pixel, patch_shape):
    img = openslide.open_slide(path)
    img = np.array(img.read_region(tl_pixel, 0, patch_shape).convert("RGB"))
    return img/255.0

In [22]:
# def get_patch2(os_obj, tl_pixel, patch_shape):
#     return np.array(os_obj.read_region(tl_pixel, 0, patch_shape).convert("RGB"))

# final_data = pd.DataFrame()
# ids = []
# for n in range(df.shape[0]):
#     img_path = "../input/mayo-clinic-strip-ai/train/"+df.iloc[n]['image_id']+".tif"
#     label = df.iloc[n]['label']
#     i_id  = df.iloc[n]['image_id']
#     ids.append(i_id)
#     img  = openslide.open_slide(img_path)
#     series = pd.Series()
#     print(n)
#     for i in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#         for j in range(img.dimensions[0]//20,img.dimensions[0]-img.dimensions[0]//20,512):
#             im = get_patch2(img , (i,j), (512,512) )
#             if im.std()>15.0:
#                 s = pd.Series(data = [(i,j)])
#                 series = series.append(s,ignore_index=True)
#                 series = series.reset_index()
#                 series = series.drop("index",axis=1)
#     final_data = pd.concat([final_data , series],axis=1)
#     final_data.columns = ids


In [23]:
def data_gen(df = df , data = final_data):
    for i in range(0,650):
        n_images = 2771 - final_data.iloc[i].isnull().sum()
        if n_images <5:
            continue
        label = df.loc[final_data.index[i]]['label']
        if label == 0 :
            images_list = np.arange(n_images)
            for x in range(3):
                final_image_list = np.random.choice(images_list, size=(5), replace=False)
                img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
                img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
                img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
                img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
                img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
                yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)
        else:
            images_list = np.arange(n_images)
            for x in range(1):
                final_image_list = np.random.choice(images_list, size=(5), replace=False)
                img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
                img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
                img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
                img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
                img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
                yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)
       
    
    
def test_gen(df = df , data = final_data):
    for i in range(650,754):
        n_images = 2771 - final_data.iloc[i].isnull().sum()
        if n_images <5:
            continue
        label = df.loc[final_data.index[i]]['label']
        if label == 0 :
            images_list = np.arange(n_images)
            final_image_list = np.random.choice(images_list, size=(5), replace=False)
            img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
            img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
            img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
            img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
            img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
            yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)
        else:
            images_list = np.arange(n_images)
            final_image_list = np.random.choice(images_list, size=(5), replace=False)
            img1 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[0]]),(512,512))
            img2 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[1]]),(512,512))
            img3 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[2]]),(512,512))
            img4 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[3]]),(512,512))
            img5 = get_patch('../input/mayo-clinic-strip-ai/train/'+final_data.index[i]+'.tif' , ast.literal_eval(final_data.iloc[i][final_image_list[4]]),(512,512))
            yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5},label)

In [24]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
     ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}, tf.float32),
    ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}, tf.TensorShape([]))
)
testset = tf.data.Dataset.from_generator(
     test_gen,
     ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}, tf.float32),
    ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}, tf.TensorShape([]))
)

In [25]:

dataset = dataset.batch(2)
dataset=dataset.prefetch(1)
dataset = dataset.shuffle(buffer_size = 10, seed=3)
testset = testset.batch(2)
testset=testset.prefetch(1)

In [26]:
efficient = tf.keras.applications.EfficientNetB7(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(512,512,3),
    pooling=None,
)
def augmentation(A):
    A = tf.keras.layers.RandomFlip()(A)
    A =tf.keras.layers.RandomRotation(
        (-0.2, 0.2),
        fill_mode='reflect',
        interpolation='bilinear',
        seed=None,
        fill_value=0.0,
    )(A)
    A = tf.keras.layers.RandomZoom(
        (-0.1,0.1),
        (-0.1,0.1),
        fill_mode='reflect',
        interpolation='bilinear',
        seed=None,
        fill_value=0.0,
    )(A)
    return A

In [33]:
inputA = keras.layers.Input(shape=(512,512,3),name='input_71')
inputB = keras.layers.Input(shape=(512,512,3),name='input_72')
inputC = keras.layers.Input(shape=(512,512,3),name='input_73')
inputD = keras.layers.Input(shape=(512,512,3),name='input_74')
inputE = keras.layers.Input(shape=(512,512,3),name='input_75')

A = augmentation(inputA)
A = efficient(A)
A = keras.models.Model(inputs=inputA, outputs=A)
B = augmentation(inputB)
B = efficient(B)
B = keras.models.Model(inputs=inputB, outputs=B)
C = augmentation(inputC)
C = efficient(C)
C = keras.models.Model(inputs=inputC, outputs=C)
D = efficient(inputD)
D = keras.models.Model(inputs=inputD, outputs=D)
E = efficient(inputE)
E = keras.models.Model(inputs=inputE, outputs=E)

X = keras.layers.Concatenate(axis=-1)([A.output, B.output, C.output, D.output, E.output])
X = keras.layers.Conv2D(1024, 4, strides=(4,4))(X)
X = keras.layers.Flatten()(X)
X = keras.layers.BatchNormalization()(X)

X = keras.layers.Dense(512 ,activation = 'relu')(X)
X = keras.layers.Dense(512 ,activation = 'relu')(X)
X = keras.layers.Dense(128 ,activation = 'relu')(X)
X = keras.layers.Dense(32 ,activation = 'relu')(X)
X = keras.layers.Dense(1 ,activation = 'sigmoid')(X)
                                               
model = keras.models.Model(inputs=[A.input,B.input,C.input,D.input,E.input], outputs=X)

In [34]:
for layer in model.layers[:-8]:
    layer.trainable = False
model.summary()

Model: "model_29"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_71 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
input_72 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
input_73 (InputLayer)           [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
random_flip_12 (RandomFlip)     (None, 512, 512, 3)  0           input_71[0][0]                   
___________________________________________________________________________________________

In [35]:
opt = tfa.optimizers.AdamW(
    weight_decay=1e-4,
    learning_rate = 0.0004,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)

In [36]:
model.compile(optimizer=opt,
              loss=loss,
                metrics=[tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), tf.keras.metrics.AUC()])

In [37]:
!mkdir model2

In [38]:
model_checkpoint_callback_LASSO = tf.keras.callbacks.ModelCheckpoint(
    filepath = './model2/model',
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
)

In [ ]:
H = model.fit(dataset, validation_data=testset, epochs=10, callbacks=[model_checkpoint_callback_LASSO])

In [ ]:
!mkdir best_model

In [ ]:
model.load_weights('./model3/model').expect_partial()
model.save('./best_model/model')